In [1]:
import numpy as np
data = np.load('data.npz')
users = data['userlist'].astype(np.int)
movies = data['movielist'].astype(np.int)
ratings = data['ratingmatrix'].astype(np.float)
moviegenre = data['genrematrix'].astype(np.int)
tagmatrix = data['tagmatrix'].astype(np.float)
tagmatrix = np.multiply(tagmatrix,100)

In [4]:
userprofilematrix = np.zeros((len(users), tagmatrix.shape[1]))
for i in range(len(users)):
    for j in range(len(movies)):
        if ratings[:,i][j] != -1:
            userprofilematrix[i] = np.add(userprofilematrix[i], tagmatrix[j])

In [8]:
from sklearn.utils.extmath import randomized_svd
U,s,V = randomized_svd(userprofilematrix, n_components=500,n_iter=20,random_state=None)

In [16]:
S = np.zeros((500,500))
np.fill_diagonal(S,s)